In [1]:
import tensorflow.keras as keras
import tensorflow.keras.backend as K
import numpy as np
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

## Keras Symbolic Example
In this example we showcase keras/tensorflow as a symbolic math toolbox that can run on cpus or gpus.
First we will create some symbolic keras tensor ```a``` as a placeholder. Then we run operations on that placeholder to generate tensors ```b, c```

In [2]:
a = keras.Input(batch_shape=(3, 3), name='Tensor_a')
b = K.dot(a, a)
c = K.exp(b)

Printing out the tensors only gives their shape and their name

In [3]:
a

<tf.Tensor 'Tensor_a:0' shape=(3, 3) dtype=float32>

Print intermediate tensor created by multiplying ```a``` by itself. note the name is autogenerated from the operation name

In [4]:
b

<tf.Tensor 'MatMul:0' shape=(3, 3) dtype=float32>

Print next intermediate tensor created by raising $e$ to the power of each element of ```b``` 

In [5]:
c

<tf.Tensor 'Exp:0' shape=(3, 3) dtype=float32>

Now I want to make a function I can give real numbers to as numpy arrays. For this use the ```K.function``` to link inputs to outputs. 

In [6]:
keras_func = K.function(inputs=[a], outputs=[c])
keras_func

Make up some dummy data to feed to our new function we just created

In [7]:
my_data = np.eye(3) * np.array([1, 2, 3])
my_data

array([[1., 0., 0.],
       [0., 2., 0.],
       [0., 0., 3.]])

Feed our dummy data to the function and print the output

In [8]:
my_output = keras_func(inputs=[my_data])
my_output

[array([[2.7182817e+00, 1.0000000e+00, 1.0000000e+00],
        [1.0000000e+00, 5.4598148e+01, 1.0000000e+00],
        [1.0000000e+00, 1.0000000e+00, 8.1030840e+03]], dtype=float32)]

See that we squared each element of our data and then raised $e$ to that power

In [9]:
np.exp(4)

54.598150033144236

In [10]:
np.exp(9)

8103.083927575384

### Gradient Example
Next we will make a symbolic gradient function. To do this we need to take the gradient with respect to a scalar. To do this we will sum our previous output ```c```.

In [11]:
scalar_output = K.sum(c)

Next we can take the gradient of that scalar with respect to our original input ```a```. Note the gradient is just another tensor object which is symboically related to our previous tensors ```a, b, c```.

In [12]:
grad = K.gradients(loss=scalar_output, variables=[a])[0]
grad

<tf.Tensor 'gradients/AddN:0' shape=(3, 3) dtype=float32>

Next we can make a function out of our gradients ```grad``` and our original input tensor ```a```. This gives us a backend function we can call using numpy arrays.

In [13]:
grad_func = K.function(inputs=[a], outputs=[grad])
grad_func

Call our function with our dummy data we made.

In [14]:
my_gradients = grad_func([my_data])
my_gradients

[array([[5.4365635e+00, 3.0000000e+00, 4.0000000e+00],
        [3.0000000e+00, 2.1839259e+02, 5.0000000e+00],
        [4.0000000e+00, 5.0000000e+00, 4.8618504e+04]], dtype=float32)]

## Placeholder dimensions
When you don't know the dimensions tensorflow will wait until you supply values. This makes things trickier as you won't know the actual dimension and will only get None until runtime. We will take our previous example but give a ```None``` dimesion

In [15]:
a = keras.Input(batch_shape=(None, 3), name='a')
b = K.dot(a, a)
c = K.exp(b)
a

<tf.Tensor 'a:0' shape=(None, 3) dtype=float32>

Note the first dimension is now a ? because tensorflow is not sure what the dimesion will be. As we print the other tensors the ? will carry through

In [16]:
b

<tf.Tensor 'MatMul_1:0' shape=(None, 3) dtype=float32>

In [17]:
c

<tf.Tensor 'Exp_1:0' shape=(None, 3) dtype=float32>

However we can run the code as before and when we give a numpy array with fixed dimensions the ? will be substituted withthe actual dimension (in this case 3) and we will get the same answer as before

In [18]:
keras_func = K.function(inputs=[a], outputs=[c])
my_data = np.eye(3) * np.array([1, 2, 3])
my_output = keras_func(inputs=[my_data])
my_output

[array([[2.7182817e+00, 1.0000000e+00, 1.0000000e+00],
        [1.0000000e+00, 5.4598148e+01, 1.0000000e+00],
        [1.0000000e+00, 1.0000000e+00, 8.1030840e+03]], dtype=float32)]